# Reforumlated Markowitz Model - Optimization Codes through Quantum Computing

Import the necessary libraries

In [181]:
import numpy as np
import pandas as pd
from dimod import ConstrainedQuadraticModel, Integer, Binary
import math
import time

Importing the Data

In [182]:
df = pd.read_excel("Cleaned_Data_Index_1.xlsx")

Changing the Date to the index

In [183]:
# Convert the 'date' column to datetime format 
df['Date'] = pd.to_datetime(df['Date'])

# Set the 'date' column as the index
df.set_index('Date', inplace=True)

Calculating the expected returns and the variance covariance matrix

In [184]:
# Calculate expected profits for each asset
expected_profits = df.mean()

# Calculate the variance-covariance matrix
cov_matrix = df.cov()


#Converting the expected profits to a numpy array
expected_profits = expected_profits.values

#The number of assets
n_assets = len(expected_profits)


### Setting the limits of the constraints

Setting the other constraints limits

In [185]:
# Maximum acceptable risk (R): The total risk that the portfolio can assume, based on variance-covariance matrix
R = 100

# Maximum units of each asset that can be purchased (D)
D = 50

# Maximum number of assets in the portfolio (K): The maximum number of different assets that can be included in the portfolio
K = 50

Creating the Decision Variables

In [186]:
y = [Binary(f"y_{i}") for i in range(n_assets)] #To represent y

In [187]:
X = [Integer(f"x_{i}") for i in range(n_assets)] #To represent x

Create the CQM Object

In [189]:
cqm = ConstrainedQuadraticModel()

Creating the Objective Function

In [190]:
H_obj = 0
for i in range(n_assets):
    H_obj -= expected_profits[i] * X[i]

Creating the risk constraint

In [191]:
# Convert the DataFrame to a NumPy array
cov_matrix_np = cov_matrix.to_numpy()

# Convert X to a NumPy array
X_numeric = np.array(X)  

# Calculate the total variance
total_variance = np.dot(X_numeric.T, np.dot(cov_matrix_np, X_numeric))

Setting the Objective Function

In [167]:
cqm.set_objective(H_obj)

Setting the Risk Constraint

In [168]:
cqm.add_constraint_from_model(total_variance, '<=', R**2, "Variance", weight=1)

'Variance'

Adding the Linking Constraints

In [169]:
# Diversification Constraints
for i in range(n_assets):
    cqm.add_constraint_from_model(X[i] - D*y[i], '<=', 0, label=f"Linking_{i}", weight=1)

Adding the cardinality constraint

In [170]:
cqm.add_constraint_from_model(sum(y), '<=', K, label=f"Cardinality", weight=1)

'Cardinality'

Solving the optimization problem

In [193]:
from dwave.system import LeapHybridCQMSampler
sampler = LeapHybridCQMSampler(token="DEV-2559c1545adea2a38a589d00b09592efde0e1bd5")  

In [172]:
# Time the sampler
start_time = time.time()
sampleset = sampler.sample_cqm(cqm)
time_taken = time.time() - start_time

In [173]:
print(sampleset.first) 

Sample(sample={'x_0': 0.0, 'x_1': 10.0, 'x_10': 0.0, 'x_100': 0.0, 'x_101': 0.0, 'x_102': 0.0, 'x_103': 0.0, 'x_104': 27.0, 'x_105': 0.0, 'x_106': 0.0, 'x_107': 0.0, 'x_108': 0.0, 'x_109': 24.0, 'x_11': 0.0, 'x_110': 18.0, 'x_111': 0.0, 'x_112': 0.0, 'x_113': 19.0, 'x_114': 0.0, 'x_115': 0.0, 'x_116': 0.0, 'x_117': 0.0, 'x_118': 50.0, 'x_119': 0.0, 'x_12': 22.0, 'x_120': 0.0, 'x_121': 6.0, 'x_122': 0.0, 'x_123': 22.0, 'x_124': 0.0, 'x_125': 0.0, 'x_126': 44.0, 'x_127': 2.0, 'x_128': 0.0, 'x_129': 0.0, 'x_13': 49.0, 'x_130': 0.0, 'x_131': 0.0, 'x_132': 0.0, 'x_133': 0.0, 'x_134': 13.0, 'x_135': 24.0, 'x_136': 37.0, 'x_137': 0.0, 'x_138': 0.0, 'x_139': 12.0, 'x_14': 0.0, 'x_140': 37.0, 'x_141': 18.0, 'x_142': 0.0, 'x_143': 48.0, 'x_144': 0.0, 'x_15': 0.0, 'x_16': 0.0, 'x_17': 0.0, 'x_18': 0.0, 'x_19': 28.0, 'x_2': 25.0, 'x_20': 1.0, 'x_21': 23.0, 'x_22': 0.0, 'x_23': 0.0, 'x_24': 0.0, 'x_25': 6.0, 'x_26': 0.0, 'x_27': 14.0, 'x_28': 0.0, 'x_29': 0.0, 'x_3': 0.0, 'x_30': 0.0, 'x_31': 0.0, 

Checking for the violation of constraints

In [174]:
for label, violation in cqm.iter_violations(sampleset.first[0], skip_satisfied=True):
    print(label, violation)

Recreating the original decision variables

In [175]:
sample = sampleset.first[0]

# Number of decision variables
num_decision_variables = n_assets

# Reconstruct the original decision variables
decision_variables = []
for i in range(num_decision_variables):
    decision_variables.append(sample[f'x_{i}'])

decision_variables


[0.0,
 10.0,
 25.0,
 0.0,
 0.0,
 20.0,
 0.0,
 20.0,
 0.0,
 0.0,
 0.0,
 0.0,
 22.0,
 49.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 28.0,
 1.0,
 23.0,
 0.0,
 0.0,
 0.0,
 6.0,
 0.0,
 14.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 45.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 9.0,
 0.0,
 0.0,
 0.0,
 0.0,
 17.0,
 0.0,
 0.0,
 0.0,
 0.0,
 17.0,
 17.0,
 0.0,
 0.0,
 29.0,
 0.0,
 1.0,
 0.0,
 0.0,
 23.0,
 0.0,
 0.0,
 19.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 20.0,
 0.0,
 0.0,
 0.0,
 0.0,
 24.0,
 47.0,
 24.0,
 0.0,
 24.0,
 5.0,
 12.0,
 0.0,
 0.0,
 9.0,
 0.0,
 0.0,
 50.0,
 0.0,
 47.0,
 0.0,
 4.0,
 34.0,
 45.0,
 0.0,
 0.0,
 0.0,
 40.0,
 0.0,
 0.0,
 0.0,
 0.0,
 27.0,
 0.0,
 0.0,
 0.0,
 0.0,
 24.0,
 18.0,
 0.0,
 0.0,
 19.0,
 0.0,
 0.0,
 0.0,
 0.0,
 50.0,
 0.0,
 0.0,
 6.0,
 0.0,
 22.0,
 0.0,
 0.0,
 44.0,
 2.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 13.0,
 24.0,
 37.0,
 0.0,
 0.0,
 12.0,
 37.0,
 18.0,
 0.0,
 48.0,
 0.0]

Calculating the total profit

In [176]:
# Multiply corresponding elements and sum them up
total = sum(x * y for x, y in zip(expected_profits, decision_variables))

print(total)

4.114048384204688


Now let us test to see if all the constraints are satisfied

In [177]:
# Extract y values from sample
y_optimized = np.array([sample[f'y_{i}'] for i in range(n_assets)])

# Convert decision_variables to a numpy array for easy manipulation
x_optimized = np.array(decision_variables)

# Now, checking the constraints with these values
# Constraint 1: Risk constraint
risk_value = np.sqrt(np.dot(np.dot(x_optimized.T, cov_matrix), x_optimized))
print(f"Risk Constraint Satisfied: {risk_value <= R}")
print(f"Risk Value: {risk_value} <= {R}")

# Constraint 2: Linking constraint
diversification_values = x_optimized <= D * y_optimized
print(f"Linking Constraints Satisfied: {all(diversification_values)}")

# Constraint 3: Cardinality constraint
cardinality_value = (x_optimized > 0).sum()
print(f"Cardinality Constraint Satisfied: {cardinality_value <= K}")
print(f"Number of Assets Included: {cardinality_value} <= {K}")

# Constraint 4: Non-negativity constraint
non_negativity_satisfied = all(x_optimized >= 0)
print(f"Non-negativity Constraints Satisfied: {non_negativity_satisfied}")


Risk Constraint Satisfied: True
Risk Value: 99.99877061087295 <= 100
Linking Constraints Satisfied: True
Cardinality Constraint Satisfied: True
Number of Assets Included: 50 <= 50
Non-negativity Constraints Satisfied: True


Checking the assets to invest in

In [178]:
values_list = x_optimized.tolist()

# Creating a new DataFrame with column names and values where values are > 0
filtered_data = {'Asset': [], 'Value': []}
for column_name, value in zip(df.columns, values_list):
    if value > 0:
        filtered_data['Asset'].append(column_name)
        filtered_data['Value'].append(value)

new_df = pd.DataFrame(filtered_data)

print(new_df)

    Asset  Value
0    AAPL   10.0
1    ABBV   25.0
2     ADI   20.0
3     ADP   20.0
4     AIZ   22.0
5     AJG   49.0
6    ALLE   28.0
7    AMAT    1.0
8     AME   23.0
9    ANET    6.0
10    AON   14.0
11   AVGO   45.0
12   BF-B    9.0
13    BKR   17.0
14    BSX   17.0
15    BWA   17.0
16    CAG   29.0
17    CAT    1.0
18   CBRE   23.0
19   CDNS   19.0
20   CHRW   20.0
21    CLX   24.0
22    CMA   47.0
23  CMCSA   24.0
24    CMG   24.0
25    CMI    5.0
26    CMS   12.0
27    COF    9.0
28   COST   50.0
29    CPB   47.0
30   CSCO    4.0
31    CSX   34.0
32   CTAS   45.0
33    CVX   40.0
34    DHI   27.0
35    DRI   24.0
36    DVA   18.0
37     ED   19.0
38     HD   50.0
39    JPM    6.0
40     KO   22.0
41    MCD   44.0
42     MO    2.0
43     PG   13.0
44   SCHW   24.0
45    STZ   37.0
46    UNH   12.0
47      V   37.0
48     VZ   18.0
49    XOM   48.0


Converting the dataframe into an excel

In [179]:
# new_df.to_excel("DF1.xlsx")

Saving the Results

In [497]:
# B = [time_taken, risk_value, R, total]
# A.append(B)

# # Create a DataFrame with the specified column names
# df = pd.DataFrame(A, columns=['time_taken', 'risk_value', 'R', 'total'])

# df.to_excel("Quantum_Results.xlsx")

Looping it all

In [194]:
# Maximum acceptable risk (R): The total risk that the portfolio can assume, based on variance-covariance matrix
Variance_Values = [10,20,30,40,50,60,70,80,90,100]

In [195]:
results_list = []  # Initialize the list

for R in Variance_Values:

    
    # Check the variance value
    print(R)

    # Create the CQM Object
    cqm = ConstrainedQuadraticModel()

    #Adding the objective function
    cqm.set_objective(H_obj)

    #Adding the risk constraint 
    cqm.add_constraint_from_model(total_variance, '<=', R**2, "Variance", weight=1)

    # Diversification Constraints
    for i in range(n_assets):
        cqm.add_constraint_from_model(X[i] - D*y[i], '<=', 0, label=f"Linking_{i}", weight=1)

    #Adding the cardinality constraint
    cqm.add_constraint_from_model(sum(y), '<=', K, label=f"Cardinality", weight=1)

    # Time the sampler
    start_time = time.time()
    sampleset = sampler.sample_cqm(cqm)
    time_taken = time.time() - start_time


    #Checking for violations
    for label, violation in cqm.iter_violations(sampleset.first[0], skip_satisfied=True):
        print(label, violation)


    #Recreating the original variables
    sample = sampleset.first[0]

    # Number of decision variables
    num_decision_variables = n_assets

    # Reconstruct the original decision variables
    decision_variables = []
    for i in range(num_decision_variables):
        decision_variables.append(sample[f'x_{i}'])


    #Calculating the total profit
    # Multiply corresponding elements and sum them up
    total = sum(x * y for x, y in zip(expected_profits, decision_variables))


    # Checking the constraints
    # Extract y values from sample
    y_optimized = np.array([sample[f'y_{i}'] for i in range(n_assets)])

    # Convert decision_variables to a numpy array for easy manipulation
    x_optimized = np.array(decision_variables)

    # Constraint 1: Risk constraint
    risk_value = np.sqrt(np.dot(np.dot(x_optimized.T, cov_matrix), x_optimized))

    # Constraint 2: Linking constraint
    diversification_values = x_optimized <= D * y_optimized

    
    # Constraint 3: Cardinality constraint
    cardinality_value = (x_optimized > 0).sum()


    # Constraint 4: Non-negativity constraint
    non_negativity_satisfied = all(x_optimized >= 0)

    
    #Creating the asset dataframe
    values_list = x_optimized.tolist()

    # Creating a new DataFrame with column names and values where values are > 0
    filtered_data = {'Asset': [], 'Value': []}
    for column_name, value in zip(df.columns, values_list):
        if value > 0:
            filtered_data['Asset'].append(column_name)
            filtered_data['Value'].append(value)

    # Create a dictionary for the current iteration
    iteration_results = {
        'time_taken': time_taken,
        'risk_value': risk_value,
        'R': R,
        'total_profit': total,
        'risk_constraint_satisfied': risk_value <= R,
        'linking_constraints_satisfied': all(diversification_values),
        'cardinality_constraint_satisfied': cardinality_value <= K,
        'non_negativity_satisfied': non_negativity_satisfied,
        'cardinality_value': cardinality_value,
        'assets_included_df': filtered_data
    }

    # Step 4: Append this dictionary to the list
    results_list.append(iteration_results)

10
20
30
40
50
60
70
80
90
100


Creating a dataframe of results

In [197]:
# Creating a DataFrame from results_list for specific metrics
metrics_columns = [
    'time_taken',
    'risk_value',
    'R',
    'total_profit',
    'risk_constraint_satisfied',
    'linking_constraints_satisfied',
    'cardinality_constraint_satisfied',
    'non_negativity_satisfied',
    'cardinality_value'
]

# Use a list comprehension to pick out these specific keys from each dictionary in the results list
final_results_data = [{col: result[col] for col in metrics_columns} for result in results_list]

# Convert this list of dictionaries to a DataFrame
final_results_df = pd.DataFrame(final_results_data)

# Save the DataFrame as an Excel file
final_results_df.to_excel('Final_Results.xlsx', index=False)


Creating the excels of Asset Allocations

In [198]:
for i, result in enumerate(results_list, start=1):
    # Convert filtered_data (which replaced new_df) to a DataFrame
    assets_df = pd.DataFrame(result['assets_included_df'])
    
    # Save the DataFrame as an Excel file
    file_name = f'Assets{i}.xlsx'
    assets_df.to_excel(file_name, index=False)